<a href="https://colab.research.google.com/github/rhiats/titanicPrediction/blob/main/titanicPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Titanic - Machine Learning from Disaster**

In [1]:
import pandas as pd

**Exploring the Training Set**

In [11]:
def duplicates(df,cols_lst):
  """
    Return df of duplicate rows across specified across columns
    https://www.statology.org/pandas-find-duplicates/

    @p:
    df (df): dataframe containing rows and columns (not empty)
    cols (list): list of columns to ck for duplicates

    @r:
    dups_df (df): Dataframe containing the duplicate rows
  """

  return df[df.duplicated(cols_lst)]

In [15]:
X=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/titanicPrediction/input/train.csv")
y=X.Survived
dupPassengers_df=duplicates(X,['PassengerId'])

In [14]:
print("The are {} duplicated passengers in the dataset".format(int(dupPassengers_df.shape[0])))

The are 0 duplicated passengers in the dataset
